<h1><center><u>Twitter Sentiment</u></center></h1>

<h5> This notebook takes a training csv file and creates a model based on positive or negative sentiment, which is then tested against a testing csv file using the saved model. </h5>

In [ ]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, SpatialDropout1D, Flatten
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import re


import seaborn as sns
print("OK")

## Load and Clean the Data
Load the data as before with pandas and take a look at the data so we know what we will be working with. 

To Do:
- Load the dataset into a dataframe with pandas
- Clean the data so it is easier to work with.

In [2]:
tweets = pd.read_csv('train_kaggle.csv', encoding = "ISO-8859-1")
tweets.head()

,Id,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [3]:
tweets['SentimentText'] = tweets['SentimentText'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]', '', x.lower()))
print(tweets['SentimentText'][0])

                     is so sad for my apl friend


## Tokenise
We are going to use tokenisation to find the 2000 most common words. Tokenising is used on a sequence of words (basically a sentence) to count their frequency. A *token* is an instance of a sequence of characters (basically a word). A type is the class of all tokens containing the same character sequence.

To Do:
- Define the max features
- Initialise a tokeniser with the max feature and a defined split
- Calculate the different word frequencies 
- Add padding to the sequences

In [6]:

vect = CountVectorizer()
x = vect.fit_transform(tweets['SentimentText'])
print(x.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Build the Model 
As before define and build the model you want to train with this dataset. 

To Do:
- Define a sequential model and add your layers. 
- Compile the model
- (Optional) Display the layers with summary()

In [7]:
embedding_dimensions = 128
lstm_out = 200 
batch_size = 32
#custom_adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model = Sequential()
model.add(Embedding(2500, embedding_dimensions,input_length = x.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
#print(model.summary())

/Users/eoinmcmahon/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/Users/eoinmcmahon/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 89252, 128)        320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 583,602
Trainable params: 583,602
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
print(tweets.size)
y = pd.get_dummies(tweets['Sentiment']).values
batch_size = 406
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 1234)
#print(X_train.shape,Y_train.shape)
#print(X_test.shape,Y_test.shape)

210000


In [9]:
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_acc', min_delta = 0.1, patience=0, verbose=2, mode='auto')


In [ ]:
# works well at epochs: 5,7, 

model.fit(X_train, Y_train, epochs=20, batch_size = batch_size, validation_data=(X_test, Y_test), shuffle=True, callbacks=[earlyStopping])

Train on 49000 samples, validate on 21000 samples
Epoch 1/20


## TESTING

model 0 --> 

In [9]:
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.49
acc: 0.76


In [10]:
model.save("bens_tweet_model.hdf5")
model = load_model("bens_tweet_model.hdf5")

In [11]:
test = pd.read_csv('test_data_kaggle.csv', encoding = "ISO-8859-1")
test.head()
test['SentimentText'] = test['SentimentText'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]', '', x.lower()))

In [12]:
max_features = 2000
tokeniser = Tokenizer(num_words=max_features, split=' ')
tokeniser.fit_on_texts(test['SentimentText'].values)
x = tokeniser.texts_to_sequences(test['SentimentText'].values)
x = pad_sequences(x, maxlen=84)
print (len(x[0]))

84


In [13]:
predictions = model.predict_classes(x)
data = {'Id' : test['Id'], 'Sentiment' : predictions}
print (data)
df = pd.DataFrame(data=data)

{'Id': 0         70013
1         70014
2         70015
3         70016
4         70017
5         70018
6         70019
7         70020
8         70021
9         70022
10        70023
11        70024
12        70025
13        70026
14        70027
15        70028
16        70029
17        70030
18        70031
19        70032
20        70033
21        70034
22        70035
23        70036
24        70037
25        70038
26        70039
27        70040
28        70041
29        70042
          ...  
29958     99971
29959     99972
29960     99973
29961     99974
29962     99975
29963     99976
29964     99977
29965     99978
29966     99979
29967     99980
29968     99981
29969     99982
29970     99983
29971     99984
29972     99985
29973     99986
29974     99987
29975     99988
29976     99989
29977     99990
29978     99991
29979     99992
29980     99993
29981     99994
29982     99995
29983     99996
29984     99997
29985     99998
29986     99999
29987    100000
Name: Id, Length:

In [14]:
df.head()

,Id,Sentiment
0,70013,1
1,70014,1
2,70015,1
3,70016,1
4,70017,1


In [15]:
df = df.astype(str)
df.to_csv('new_output.csv', index=False)

In [16]:
df.dtypes

Id           object
Sentiment    object
dtype: object